# Coordinated Multipoint Using a Quantum Computer
This notebook demonstrates

# What is Coordinated Multipoint?

TBD: add description, papers, etc

# Modeling a Network

TBD: explain the King's graph-based network model

## Create a Network Graph

TBD: explain the King's graph-based network model

In [ ]:
# This code imports from "helpers", a folder colocated with this Jupyter Notebook.
from helpers.network import configure_network, print_network_stats

network, _ = configure_network(lattice_size=4, ratio=1)
num_tx, num_rx = print_network_stats(network)

The next code cell plots the network graph, showing transmitters in red and receivers in green. 

In [ ]:
import matplotlib.pyplot as plt
from helpers.draw import draw_network
%matplotlib inline     

draw_network(network)

## Create Channels

Between the transmitters and receivers there are channels etc

In [ ]:
import numpy as np
import dimod

channels, channel_power = dimod.generators.mimo.create_channel(num_receivers=num_rx, num_transmitters=num_tx, F_distribution=("binary", "real"))

print(f"`channels` is a {channels.shape[0]} by {channels.shape[1]} matrix.")

tx = np.random.choice(list(range(num_tx)))
rx = np.random.choice(list(range(num_rx)))
print(f"The value of the channel between transmitter {tx} and receiver {rx} is {channels[rx, tx]}.")  

# Decoding Signals

TBD: explain linear filters

## Create a Filter
Explain about filters

In [ ]:
from helpers.filters import create_filter

filter_methods = ['zero_forcing', 'matched_filter', 'MMSE']
filters = {f'filter_{method}': create_filter(channels, method=method) for method in filter_methods}

## Simulated a Transmitted Signal


In [ ]:

transmitted_symbols = np.random.choice([1, -1], size=[num_tx, 1]) 
y, _, _, _ = dimod.generators.mimo._create_signal(channels, transmitted_symbols=transmitted_symbols)
print(y.flatten())

## Decode a Transmitted Signal

In [ ]:
from helpers.filters import apply_filter

v = {name: apply_filter(filter, y) for name, filter in filters.items()}
success_rate = {name: round(100*sum(v[name].flatten() == transmitted_symbols.flatten())/num_tx) for name, filter in filters.items()}    

for method in filters:
    print(f"{method}: decoded with a success rate of {success_rate[method]}%.")

# Scaling Up

In [ ]:
import networkx as nx
import time

for lattice_size in [5, 10, 15]:

    network, _ = configure_network(lattice_size=lattice_size, ratio=1)
    num_tx = sum(nx.get_node_attributes(network, "num_transmitters").values())
    num_rx = sum(nx.get_node_attributes(network, "num_receivers").values())
    channels, channel_power = dimod.generators.mimo.create_channel(num_receivers=num_rx, num_transmitters=num_tx, F_distribution=("binary", "real"))
    print(f"\nFor a network of {num_tx} cellphones and {num_rx} base stations:\n")
    for method in filter_methods:
        start_t = time.time_ns()
        create_filter(channels, method=method)
        time_ms = (time.time_ns() - start_t)/1000000
        print(f"\t* {method} took about {round(time_ms)} milliseconds.")
    

# Solving with a Quantum Computer
Some words here

In [ ]:
from dwave.system import DWaveSampler

qpu = DWaveSampler(solver=dict(topology__type="pegasus"))

print(f"Selected {qpu.solver.name} with {len(qpu.nodelist)} qubits.")

## Representing the Problem as a BQM

### Create a Network

In [ ]:
network, _ = configure_network(lattice_size=16, ratio=1, qpu=qpu)
num_tx, num_rx = print_network_stats(network)
channels, channel_power = dimod.generators.mimo.create_channel(num_receivers=num_rx, num_transmitters=num_tx, F_distribution=("binary", "real"))

### Create Filters and a Signal

In [ ]:
filter_mf = create_filter(channels, method='matched_filter')
transmitted_symbols = np.random.choice([1, -1], size=[num_tx, 1]) 
y, _, _, _ = dimod.generators.mimo._create_signal(channels, transmitted_symbols=transmitted_symbols)


## Create a BQM

In [ ]:
bqm = dimod.generators.mimo.spin_encoded_comp(network, 
                                              modulation = 'BPSK', 
                                              transmitted_symbols=transmitted_symbols, 
                                              F_distribution=('binary','real'), 
                                              SNRb=float('inf'),
                                              F=channels,
                                              y=y)

## Decode a Transmitted Signal